# Word2Vec
- Skip Gram

## Import Packages

In [1]:
# import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
from tqdm import tqdm_notebook
from nltk.corpus import stopwords

Instructions for updating:
non-resource variables are not supported in the long term


## Configurations

In [49]:
hyper = {
    "emb_size":100,
    "window_size": 5,
    "min_count":1,
    "sg":0,
    "negative_size": 5,
    "ns_exponent": 0.75,
    "cbow_mean":1,
    "iter":10, #epochs
    "learning_rate":0.1, # initial learning rate
    "sample": 0.00001, #defualt=0.001
#     "vocab_size":0,
    "batch_size":512,
    "update_size":16,
}

In [3]:
stopwords_en = stopwords.words('english')

## Load Dataset
- IMDB 영화 리뷰 데이터
    - 텍스트 분류 중 감정 분류 모델 연습용으로 쓰임
    - Label: 영화 리뷰가 긍정이면 1, 부정이면 0
- word embedding을 생성하는데 적절한 데이터는 아닌것 같지만 임시로 사용한다.

In [4]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(
    num_words=None, # max number of words to include
    skip_top=0, # skip the top N most frequently occurring words
)
print(f"X_train: {x_train.shape}, y_train: {y_train.shape}")
print(f"X_test: {x_test.shape}, y_test: {y_test.shape}")

X_train: (25000,), y_train: (25000,)
X_test: (25000,), y_test: (25000,)


In [5]:
print(f"The num of categories: {len(set(y_train))}")

The num of categories: 2


In [6]:
word2index = tf.keras.datasets.imdb.get_word_index()
index2word = {}
for word, index in word2index.items() :
    index2word[index] = word

print(f"The num of words: {len(word2index)}")
word2index["woods"], index2word[1408]

The num of words: 88584


(1408, 'woods')

In [7]:
len(word2index.values())

88584

### Change from index to word

In [8]:
def change_from_index_to_word(index_list) :
    word_list = []
    for index in index_list :
        if index in index2word :
            word_list.append(index2word[index])
        else :
            print(f"{index} is not exist in index2word!")
    return word_list

In [9]:
train_texts = []
for indexs in x_train :
    train_texts.append(change_from_index_to_word(indexs))
len(train_texts)

88585 is not exist in index2word!
88586 is not exist in index2word!


25000

## Functions

In [10]:
def get_vocabulary(texts, min_count) :

    # Calculate frequency of word appearance.
    word2count = {}
    for text in texts :
        for word in text :
            if word in word2count :
                word2count[word] += 1
            else :
                word2count[word] = 1
    
    # Save only words that meet min_count condition.
    vocab = [word for word in word2count if word2count[word] >= min_count]
    filtered_word2count = {}
    for word in vocab :
        filtered_word2count[word] = word2count[word]
    print(f"The num of words that meet min_count >= {min_count}: {len(vocab):,}")
    
    return filtered_word2count, vocab

In [11]:
min_count = hyper.get("min_count")
word2count, vocab = get_vocabulary(train_texts, min_count)

The num of words that meet min_count >= 1: 88,583


In [50]:
hyper["vocab_size"] = len(vocab)

In [35]:
# def generate_word_pairs(texts, word2count, vocab, hyper) :
#     '''
#     return [[target word(words)], [context word(words)], [negative word(word2)]]
#     '''
#     sample = hyper.get("sample")
#     window_size = hyper.get("window_size")
#     negative_size = hyper.get("negative_size")
#     ns_exponent = hyper.get("ns_exponent")
    
#     filtered = 0
#     sampled = 0
#     training_data = []
#     negative_prob = np.array(list(word2count.values()))
#     negative_prob = (negative_prob ** ns_exponent) / np.sum(negative_prob ** ns_exponent)
    
#     for raw_text in tqdm_notebook(texts) :
#         text = [word for word in raw_text if (word in vocab)&(word not in stopwords_en)]
        
#         if len(text) > 1 :
#             for i in range(len(text)) :
#                 # get target word
#                 target_word = text[i]
#                 target_index = vocab.index(text[i])
                
#                 # probability of excluding word from learning
#                 prob = 1 - np.sqrt(sample / word2count[target_word])
#                 if np.random.rand() < prob :
#                     sampled += 1
#                     continue
                        
#                 # get context words
#                 context_words = []
#                 context_indexs = []
#                 for j in range(i - window_size if i - window_size > 0 else 0, i + window_size if i + window_size <= len(text) - 1 else len(text) - 1) :
#                     context_words.append(text[j])
#                     context_indexs.append(vocab.index(text[j]))
                    
#                 # get negative words
#                 negative_words = list(np.random.choice(vocab, p=negative_prob, size=negative_size))
#                 negative_indexs = [vocab.index(negative_word) for negative_word in negative_words]
                
#                 # append rows
#                 training_data.append([target_index, context_indexs, negative_indexs])
#         else :
#             filtered += 1
                
#     print(f"- The final training data: {len(training_data)}.")
#     print(f"- Filtered {filtered} texts.")
#     print(f"- Sampled {sampled} texts.")
#     return training_data

In [74]:
def generate_word_pairs(texts, word2count, vocab, hyper) :
    '''
    return [[target word, context word],[target word, context word],...]
    '''
    sample = hyper.get("sample")
    window_size = hyper.get("window_size")
    
    filtered = 0
    sampled = 0
    training_data = []

    for raw_text in tqdm_notebook(texts) :
        text = [word for word in raw_text if (word in vocab)&(word not in stopwords_en)]
        
        if len(text) > 1 :
            for i in range(len(text)) :
                # get target word
                target_word = text[i]
                target_index = vocab.index(text[i])
                
                # probability of excluding word from learning
                prob = 1 - np.sqrt(sample / word2count[target_word])
                if np.random.rand() < prob :
                    sampled += 1 * 2 * window_size
                    continue
                        
                # get context words
                for j in range(i - window_size if i - window_size > 0 else 0, i + window_size if i + window_size <= len(text) - 1 else len(text) - 1) :
                    context_word = text[j]
                    context_index = vocab.index(context_word)
                    
                    # append rows
                    training_data.append([target_index, context_index])
        else :
            filtered += 1 * 2 * window_size
                
    print(f"- The final training data: {len(training_data)}.")
    print(f"- Filtered {filtered} texts.")
    print(f"- Sampled {sampled} texts.")
    return training_data

In [ ]:
training_data = generate_word_pairs(train_texts, word2count, vocab, hyper)

In [ ]:
training_data[:5]

## Define Model and Run

In [ ]:
window_size = hyper.get("window_size")
negative_size = hyper.get("negative_size")
batch_size = hyper.get("batch_size")
update_size = hyper.get("update_size")
vocab_size = hyper.get("vocab_size")
emb_size = hyper.get("emb_size")
learning_rate = hyper.get("learning_rate")
epochs = hyper.get("iter")
    
target = tf.placeholder(dtype=tf.int32, shape=[batch_size])
context = tf.placeholder(dtype=tf.int32, shape=[batch_size, 1])

W = tf.Variable(tf.random_uniform([vocab_size, emb_size], -1.0, 1.0))
target_emb = tf.nn.embedding_lookup(W, target)

weights = tf.Variable(tf.random_uniform([vocab_size, emb_size], -1.0, 1.0))
biases = tf.Variable(tf.zeros([vocab_size]))

loss = tf.reduce_mean(tf.nn.nce_loss(weights, biases, context, target_emb, update_size, vocab_size))
train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss)
init = tf.global_variables_initializer()

In [ ]:
with tf.Session() as session:
    session.run(init)
    for epoch in range(epochs) :
        
        target_words = []
        context_words = []
        for i in range(1, len(training_data)+1) :
            target_words.append([training_data[i][0]])
            context_words.append([training_data[i][1]])
            if i % batch_size == 0 :
                session.run(
                    train, 
                    feed_dict = {
                        target:target_words,
                        context:context_words,
                    }
                )
        
    # Get W, U, Total-Loss
    weight, weight_ = session.run([W, U])
    print(weight.shape)
    print(weight_.shape)